In [1]:
import scanpy as sc
import numpy as np
import pandas as pd


In [4]:
adata_obs_meta = pd.read_csv('adata_obs_meta.csv')
id_map = pd.read_csv('id_map.csv')
multiome_obs_meta = pd.read_csv('multiome_obs_meta.csv')
multiome_var_meta = pd.read_csv('multiome_var_meta.csv')
sample_submission = pd.read_csv('sample_submission.csv')


### 一、总览

In [11]:
#1.adata_obs_meta
len(pd.DataFrame(adata_obs_meta['library_id'].value_counts()))
    #48正好对应上library的定义。一共6个板，每个板每行一个library
    

48

In [10]:
#2.id_map
id_map['cell_type'].value_counts()
    #这边只提到了两种细胞，因为测试集只有两种细胞（那张蓝色的图）
    #但是数量没对上，介绍里说测试集分别有50和79个化合物
    #介绍里说这些id是要被预测的，所以意思应该是不一定要全预测，只要预测它给的
    

B cells          128
Myeloid cells    127
Name: cell_type, dtype: int64

In [13]:
#3.sample_submission
sample_submission
    #列名是multiome_var_meta里的location列（一些基因名）
    

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
multiome_var_meta['feature_type'].value_counts()
    #可以看到，有四千多个基因不在sample_submission里
    

Peaks              135418
Gene Expression     22787
Name: feature_type, dtype: int64

In [3]:
#4.adata_train
adata_train = pd.read_parquet("adata_train.parquet")
adata_train['gene'].value_counts()
    #这里的基因数也和前面的不等
    #obs_id是细胞的id
    

B2M           240088
RPL41         240087
MT-RNR2       240086
TPT1          240085
MALAT1        240082
               ...  
RGS7               1
TAL1               1
AC105235.1         1
AL157895.1         1
AC129502.1         1
Name: gene, Length: 21255, dtype: int64

In [5]:
#5.de_train
de_train = pd.read_parquet("de_train.parquet")
de_train['cell_type'].value_counts() #和蓝色的图能对上，少了4个CD8+原因也有讲


NK cells              146
T cells CD4+          146
T regulatory cells    146
T cells CD8+          142
B cells                17
Myeloid cells          17
Name: cell_type, dtype: int64

In [33]:
de_train['control'].value_counts()
#两个positive control，六个细胞类型，所以是12个。和蓝色的图也对上了


False    602
True      12
Name: control, dtype: int64

In [28]:
#6.multiome_train
multiome_train = pd.read_parquet("multiome_train.parquet")


In [29]:
multiome_train


,obs_id,location,count,normalized_count
0,000225c1151ab841,AAMP,1,6.320659
1,000225c1151ab841,AASS,1,6.320659
2,000225c1151ab841,ABCC11,1,6.320659
3,000225c1151ab841,ABCC2,1,6.320659
4,000225c1151ab841,ABR,1,6.320659
...,...,...,...,...
216251363,ffffeba9dbf2bab1,chrY:19499847-19500746,1,3.342186
216251364,ffffeba9dbf2bab1,chrY:19566990-19567754,1,1.154666
216251365,ffffeba9dbf2bab1,chrY:20735890-20736626,3,7.049924
216251366,ffffeba9dbf2bab1,chrY:21260054-21260842,2,3.967924


## 方法一：用label encoder来处理数据集

### 1、预处理

In [6]:
sum(de_train.isnull().sum()) #没有空值
    #还看到她们画了图，筛选别的变量的空值什么的，因为最终啥也没删掉，我就没写进来
    

0

In [6]:
features = ['cell_type','sm_name','sm_lincs_id','SMILES','control']
labels_train = de_train.drop(columns = features)
features_train = de_train.iloc[:,:2]


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ct = le.fit_transform(features_train['cell_type'])
sm = le.fit_transform(features_train['sm_name'])

features_train_encoded = pd.DataFrame({'cell_type_encoded':ct,'sm_name_encoded':sm})
#原文用的是ordinal encoder，但是我觉得没必要


In [ ]:
# 一种是像这里一样，只对features转成labelencoder；
# 一种是对features转成onehot encoder；
# 还有一种是用上了pca（不过我得去回顾一下怎么做的）


In [63]:
features_train_encoded.to_csv("features_train_encoded.csv")
labels_train.to_csv("labels_train.csv")

In [9]:
features_train_encoded = pd.get_dummies(features_train)

In [10]:
features_train_encoded.to_csv("features_train_onehotEncoded.csv")

### 2、处理test数据集


In [8]:
df = pd.concat([features_train_encoded,features_train],axis=1)

In [9]:
#对应关系的展示
cell_type_mapping = df.groupby(['cell_type_encoded', 'cell_type']).size().reset_index(name='count')
cell_type_mapping

,cell_type_encoded,cell_type,count
0,0,B cells,17
1,1,Myeloid cells,17
2,2,NK cells,146
3,3,T cells CD4+,146
4,4,T cells CD8+,142
5,5,T regulatory cells,146


In [10]:
sm_name_mapping = df.groupby(['sm_name_encoded', 'sm_name']).size().reset_index(name='count')
sm_name_mapping

,sm_name_encoded,sm_name,count
0,0,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,4
1,1,ABT-199 (GDC-0199),4
2,2,ABT737,4
3,3,AMD-070 (hydrochloride),4
4,4,AT 7867,4
...,...,...,...
141,141,Vandetanib,4
142,142,Vanoxerine,4
143,143,Vardenafil,4
144,144,Vorinostat,4


In [11]:
cell_type_mapping_dict = dict(zip(cell_type_mapping['cell_type'], cell_type_mapping['cell_type_encoded']))
sm_name_mapping_dict = dict(zip(sm_name_mapping['sm_name'], sm_name_mapping['sm_name_encoded']))

In [12]:
id_map['cell_type'] = id_map['cell_type'].map(cell_type_mapping_dict)

In [13]:
id_map['sm_name'] = id_map['sm_name'].map(sm_name_mapping_dict)

In [14]:
id_map

,id,cell_type,sm_name
0,0,0,0
1,1,0,1
2,2,0,2
3,3,0,3
4,4,0,4
...,...,...,...
250,250,1,141
251,251,1,142
252,252,1,143
253,253,1,144


In [27]:
sample_submission = sample_submission.iloc[:,1:]
sample_submission_encoded = pd.concat([id_map, sample_submission],axis=1)
sample_submission_encoded

,id,cell_type,sm_name,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,1,141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,1,142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,1,143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,1,144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
sample_submission_encoded.to_csv("sample_submission_encoded.csv", index=False)

## 方法二： 用one hot encoder来处理分类型变量¶

In [9]:
#训练集：features_train
#测试集：id_map
#拼一起同时转换，再切开就好了
id_map = id_map.iloc[:,1:]
data = pd.concat([features_train,id_map],axis=0)

In [11]:
data_encoded = pd.get_dummies(data)

In [14]:
train_data_encoded = data_encoded.iloc[:614,:]
test_data_encoded = data_encoded.iloc[614:,:]

In [18]:
train_data_encoded.to_csv("features_train_onehotEncoded.csv")
test_data_encoded.to_csv("sample_submission_onehotEncoded.csv")